# Task : Identifying the number of duplicate questions

Importing Training and test data set

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import time
import numpy as np

In [ ]:
dataset = pd.read_csv('corpusTrain.csv')
data_test = pd.read_csv('corpusTest.csv')

In [ ]:
for i in range(10):
    print(dataset.Content[i])

In [ ]:
for i in range(10):
    print(dataset.Content[i])

In [ ]:
questions = dataset.iloc[:,1]

In [ ]:
qtest = data_test.iloc[:,1]

In [ ]:
qtest = qtest.apply(lambda x : (re.sub('[^a-zA-Z]', ' ', x)).lower())

In [ ]:
questions = questions.apply(lambda x : (re.sub('[^a-zA-Z]', ' ', x)).lower())

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(questions)
sparse=vectorizer.transform(questions)
sparse_test=vectorizer.transform(qtest)

memory error occured at 100k after 10 k ran smoothly

now i try to split the sparse matrix

In [ ]:
y=np.arange(0,51).tolist()
y = [k*10000 for k in y]
y.append(sparse.shape[0])

In [ ]:
countn = 0
t_startn = time.time()
for t in range(sparse_test.shape[0]):
    for k in range(len(y)-1):    
        cm=cosine_similarity(sparse_test[t],sparse[y[k]:y[k+1]])
        if np.max(cm) > 0.8:
            countn += 1
            break
query_time1 = time.time() - t_startn

In [ ]:
countn

In [ ]:
query_time1

implementing jaccard similarity
consider the fact that jaccard_similarity = 1 - jaccard distance metric
since the pairwise implementation for the jaccard metric cannot take sparse matrices, we had to convert to numpy and to do this
we needed a smaller array size so we reduced the feature size of the vectorizer

In [ ]:
vectorizer1 = TfidfVectorizer()
sparse1=vectorizer1.fit_transform(questions)
sparse_test1=vectorizer1.transform(qtest)

In [ ]:
sp = sparse_test1.todense()

In [ ]:
y1=np.arange(0,51).tolist()
y1 = [k*1000 for k in y1]
y1.append(sparse1.shape[0])

In [ ]:
from sklearn.metrics import pairwise_distances
countj = 0
t_startj = time.time()
for t in range(sparse_test1.shape[0]):
    print(t)
    for k in range(len(y1)-1):  
        spt = sparse1[y1[k]:y1[k+1]].todense()
        dist = pairwise_distances(sp[t],spt,metric='jaccard')
        if 1 - np.min(dist) > 0.8:
            countj += 1
            break
query_timej = time.time() - t_startj

In [ ]:
countj

In [ ]:
#deducting wasted time
tc=time.time()
print(1)
ttp=time.time()-tc
tcc=time.time()
spk = sparse[:10000].todense()
ttd = time.time()-tcc

In [ ]:
print ('query time is : {} hours'.format((query_timej - sparse_test.shape[0]*ttp - sparse_test.shape[0]*len(y)*ttd)/3600))

implementing the datasketch library

In [ ]:
from datasketch import MinHash, MinHashLSH

In [ ]:
qsplit = qtest.apply(lambda x : set(x.split()))
questions_split = questions.apply(lambda x : set(x.split()))

In [ ]:
init=time.time()
#minhash table for trainx
mhashes2 =[]
for i in range(questions_split.shape[0]):
    mhashes2.append(MinHash(num_perm=16))
    for d in questions_split[i]:
        mhashes2[i].update(d.encode('utf8'))
#creating lsh index
lsh = MinHashLSH(threshold=0.8, num_perm=16)
for i in range(questions_split.shape[0]):
    lsh.insert('t'+str(i), mhashes2[i])
Build_time=time.time() - init

In [ ]:
Build_time

Querying

In [ ]:
count_num = 0
init1=time.time()
#minhash index for test set
mhashes =[]
for i in range(qsplit.shape[0]):
    mhashes.append(MinHash(num_perm=16))
    for d in qsplit[i]:
        mhashes[i].update(d.encode('utf8'))
for i in range(qsplit.shape[0]):
    if len(lsh.query(mhashes[i])):
        count_num += 1
q_time=time.time() - init1

In [ ]:
q_time

In [ ]:
count_num

In [ ]:
#n permutations = 32
init=time.time()
#minhash table for trainx
mhashes2 =[]
for i in range(questions_split.shape[0]):
    mhashes2.append(MinHash(num_perm=32))
    for d in questions_split[i]:
        mhashes2[i].update(d.encode('utf8'))
#creating lsh index
lsh1 = MinHashLSH(threshold=0.8, num_perm=32)
for i in range(questions_split.shape[0]):
    lsh1.insert('t'+str(i), mhashes2[i])
Build_time32=time.time() - init

In [ ]:
Build_time32

In [ ]:
count_num32 = 0
init1=time.time()
#minhash index for test set
mhashes =[]
for i in range(qsplit.shape[0]):
    mhashes.append(MinHash(num_perm=32))
    for d in qsplit[i]:
        mhashes[i].update(d.encode('utf8'))
for i in range(qsplit.shape[0]):
    if len(lsh1.query(mhashes[i])):
        count_num32 += 1
q_time1=time.time() - init1

In [ ]:
q_time1

In [ ]:
count_num32

In [ ]:
#n permutations = 64
init=time.time()
#minhash table for trainx
mhashes2 =[]
for i in range(questions_split.shape[0]):
    mhashes2.append(MinHash(num_perm=64))
    for d in questions_split[i]:
        mhashes2[i].update(d.encode('utf8'))
#creating lsh index
lsh2 = MinHashLSH(threshold=0.8, num_perm=64)
for i in range(questions_split.shape[0]):
    lsh2.insert('t'+str(i), mhashes2[i])
Build_time64=time.time() - init

In [ ]:
Build_time64

In [ ]:
count_num64 = 0
init1=time.time()
#minhash index for test set
mhashes =[]
for i in range(qsplit.shape[0]):
    mhashes.append(MinHash(num_perm=64))
    for d in qsplit[i]:
        mhashes[i].update(d.encode('utf8'))
for i in range(qsplit.shape[0]):
    if len(lsh2.query(mhashes[i])):
        count_num64 += 1
q_time2=time.time() - init1

In [ ]:
q_time2

In [ ]:
count_num64

Implementing Random forest using single hash table. the accuracy here can be improved with the use of 
multiple hashtables. it is worth noting here that facing memory issueas here we also reduced the max_feature of the vectorizer

In [ ]:
vectorizer2 = TfidfVectorizer()
vectorizer2.fit(questions)
sparse2=vectorizer2.transform(questions)
sparse2_test=vectorizer2.transform(qtest)
sp=sparse2
spt=sparse2_test

In [ ]:
def hashf(vects,proj):
    h=vects.dot(proj.T)>0
    return bool_int(h[0])

In [ ]:
def bool_int(b):   
    y = 0
    for i,j in enumerate(b):
        if j: y += 1<<i
    return y
#bool_int([1,0,0,1])

In [ ]:
import scipy

In [ ]:
from sklearn.metrics.pairwise import *
import scipy
class LSH:
    
    def __init__(self, hash_size, dim,thr):
        self.table = dict()
        self.thr = thr
        self.hash_size = hash_size
        self.projections = np.random.randn(self.hash_size, dim)

    def insert(self, vecs):
        h= hashf(vecs, self.projections)
        if h in self.table.keys():
            self.table[h] = scipy.sparse.vstack((self.table[h],vecs))
        else:
            self.table[h] = vecs

    def query(self, vecs):
        h = hashf(vecs, self.projections)
        cm=cosine_similarity(vecs,self.table[h])
        if np.max(cm)>self.thr:
            return True
        return False

In [ ]:
lsh = LSH(2,sp.shape[1],0.8)

In [ ]:
t_init = time.time()
for i in range(sp.shape[0]):
    lsh.insert(sp[i])
    #print(i)
#lsh.sparse()
ent = time.time() - t_init

In [ ]:
ent

In [ ]:
print('hash distribution')
for h in lsh.table.keys():
    print('{}:{}'.format(h,lsh.table[h].shape[0]))

the search space has now been divided into four but as we can see , each segment is still pretty large so we do not expect 
query time to be small

In [ ]:
cntt=0
qtt=time.time()
for i in range(spt.shape[0]): 
    if lsh.query(spt[i]):
        cntt+=1
qte = time.time() - qtt

In [ ]:
cntt

In [ ]:
qte

In [ ]:
#for k=3
lsh2 = LSH(3,sp.shape[1],0.8)

In [ ]:
t_init2 = time.time()
for i in range(sp.shape[0]):
    lsh2.insert(sp[i])
ent2 = time.time() - t_init2

In [ ]:
ent2

In [ ]:
print('hash distribution')
for h in lsh2.table.keys():
    print('{}:{}'.format(h,lsh2.table[h].shape[0]))

In [ ]:
cntt2=0
qtt=time.time()
for i in range(spt.shape[0]):
    if lsh2.query(spt[i]):
        cntt2+=1
qte2 = time.time() - qtt

In [ ]:
qte2

In [ ]:
cntt2